In [1]:
import glob
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

from moviepy.editor import VideoFileClip
#from IPython.display import HTML

#Returns the objpoints and imgpoints needed for undistortion
def calibrate( images, nx = 6, ny = 9 ):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((nx*ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Step through the list and search for chessboard corners
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            
    return objpoints, imgpoints

#Uses the objpoints and imgponts from calibrate to undistort an image
def undistort( img, objpoints, imgpoints ):
    # Get the image size
    img_size = (img.shape[1], img.shape[0])

    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)

    #Undistort the image
    img_undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    return img_undist

#Makes the image a single channel image
#Can do gray, R, G, B, H, L, ans S
def color_select(img, gray = False, hls=True, channel = 0):
    # Return the gray channel if requested
    if(gray):
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        return img 
    # Convert to HLS color space if requested
    if (hls):
        img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

    # Return the selected channel
    channelImage = img[:,:,channel]
    
    return channelImage


#Does sobel 'x', 'y', 'mag', or 'dir' for rgb and grayscale images
def sobel(img, type='x', sobel_kernel=3, rgb = True):
    # Convert to grayscale, if necessary
    if rgb:
        gray = color_select(img, True)
    else:
        gray = image
        
    #Take the derivative in x and y 
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    #Decide which sobel to return
    if type == 'x':
        sobel_image = sobelx
    elif type == 'y':
        sobel_image = sobely
    elif type =='mag':
        sobel_image = np.sqrt(np.power(sobelx, 2) + np.power(sobely, 2))
    elif type == 'dir':
        sobel_image = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    else:
        return (none)
        
    # Always return a scaled image - 0 to 255
    abs_sobel = np.absolute(sobel_image)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))

    return scaled_sobel

# Thresholds a single channel image given an exclusive lower bound (>)
# and inclusive upper (<=)
def thresh(img, thresh=(0, 255)):
    #Apply a threshold to the image channel
    binary = np.zeros_like(img)
    binary[(img > thresh[0]) & (img <= thresh[1])] = 1
    # 3) Return a binary image of threshold result
    #binary_output = np.copy(binary) # placeholder line
    return binary

#Define the pipeline to get to a thresholded image
def thresh_pipeline( image ):

    #Threshold the R channel
    rgb0 = color_select(image, hls=False, channel = 0)
    rgb0_thresh = thresh(rgb0, thresh=(224, 255))

    #Threshold the G channel
    rgb1 = color_select(image, hls=False, channel = 1)
    rgb1_thresh = thresh(rgb1, thresh=(200, 255))

    #Threshold the L channel
    hls1 = color_select(image, hls=True, channel = 1)
    hls1_thresh = thresh(hls1, thresh=(192, 255))

    #Threshold the B channel
    hls2 = color_select(image, hls=True, channel = 2)
    hls2_thresh = thresh(hls2, thresh=(192, 255))

    all_colors = rgb0_thresh+rgb1_thresh+hls1_thresh+hls2_thresh
    
    #cv2.imwrite("image_all_colors.jpg", all_colors*64)

    #Threshold the X sobel
    grad_x = sobel(image, type='x', sobel_kernel=5) # , thresh_min=5, thresh_max=100,
    grad_binary_x = thresh(grad_x, thresh=(32, 192))

    #Threshold the Y sobel
    grad_y = sobel(image, type='y',  sobel_kernel=5)
    grad_binary_y = thresh(grad_y, thresh=(32, 192))

    #Threshold the sobel magnitude
    grad_mag = sobel(image, type='mag', sobel_kernel=5) # , thresh_min=5, thresh_max=100,
    grad_binary_mag = thresh(grad_mag, thresh=(64, 255))

    #Threshold the sobel direction
    grad_dir = sobel(image, type='dir',  sobel_kernel=5)
    grad_binary_dir = thresh(grad_dir, thresh=(128, 255))

    all_grad = grad_binary_dir+grad_binary_mag+grad_binary_x+grad_binary_y

    #cv2.imwrite("image_all_grad.jpg", all_grad*64)

    #Color info is weighted 3x to gradient info
    all_data = all_grad + all_colors*3

    return all_data

#Calculate the warping matrix
def calcwarp(img, size = 300):
    height = img.shape[0] # 720
    width = img.shape[1] # 1280

    #This is the midpoint in the x direction
    xm = width/2

    #This is the percentage of the image at the 'farthest point'
    xtp = 0.079 # larger means narrower

    #Calculate the points of the image at the 'farthest point'
    xtl = xm - xm * xtp
    xtr = xm + xm * xtp

    #Set the points of the image at the 'nearest point'
    xbl = 0 #640 - 640 * xbp
    xbr = width 

    #Hardcoded vertical section to take; crops the bottom 10% and top 60%
    yb = 650 #height*.9 # 648 
    yt = 440 #height*.61  # 440 

    #Determine the source and dest vertices
    vertices = [[(xtl, yt), (xtr, yt), (xbl, yb), (xbr, yb)]]
    src_vertices = np.array(vertices, dtype=np.float32)
    dest_vertices = np.float32([[0,0],[size,0],[0,size],[size,size]])

    #Get the warpping matrix and the inverse warping matrix
    M = cv2.getPerspectiveTransform(src_vertices, dest_vertices)
    Minv = np.linalg.inv(M)

    return M, Minv

def warp(img, M, size): 
    #Warp and return the image per the transformation matrix
    warped = cv2.warpPerspective(img, M, size)
    return warped


def histogram_points(image, pixelsperbin=10):
    height = image.shape[0] # 720
    width = image.shape[1] # 1280

    #Get a histogram of the threshold
    #nbins = width/pixelsperbin
    histogram = np.sum(image[height*2/3:,:], axis=0)
    values = np.asarray(list(range(0, width)))

    #Split into left and right halves of histogram
    l_histogram = histogram[0:width/2]
    r_histogram = histogram[width/2:]

    #Split values into left and right
    l_values = values[0:width/2]
    r_values = values[width/2:]

    #Check to make sure we don't have an empty matrix...
    l_sum = np.sum(l_histogram)
    r_sum = np.sum(r_histogram)

    #Get the average of the left and right halves, weighted by the number of hits
    if(l_sum > 0):
        l_max_index = int(np.average(l_values, weights=l_histogram))
    else:
        l_max_index = 0

    if(r_sum > 0):
        r_max_index = int(np.average(r_values, weights=r_histogram))
    else:
        r_max_index = 0

    return l_max_index, r_max_index

def run_fit(image, fit, n = 100, margin = 25, left = 0, right = 299):

    #Hold on to this for smoothing
    old_fit = fit

    height = image.shape[0] # 300
    width = image.shape[1] # 300

    #Lists to hold the points to fit to
    x_list = []
    y_list = []

    #For all Y
    for y in range(height-1, -1, -1):
        #Calculate X
        x = int(round(fit[0]*y**2 + fit[1]*y + fit[2]))

        #Descending margin with decreasing y
        #More likely to find the lane at the bottom
        #Less likely to get creative at the top
        margin = int(y/6) #50 to 0 for 300 to 0

        #Calculate a region of pixels to grab
        x_min = x-margin
        x_max = x+margin

        #Error check the region of pixels
        x_min = max(x_min, left)
        x_min = min(x_min, right)


        x_max = max(x_max, left)
        x_max = min(x_max, right)

        #Grab the region of pixels
        image_sect = image[y,x_min:x_max]

        #Get the x coords of those pixels
        x_sect = np.where(image_sect==1)
        x_sect = x_sect + np.ones_like(x_sect)*x_min

        #And the y coords
        y_sect = np.ones_like(x_sect)*y

        #And add them both to their respective lists
        x_list.extend(x_sect.tolist()[0])
        y_list.extend(y_sect.tolist()[0])

        #If we have a certain number of points
        if( len(x_list) > n):
            #Use the new 'lane line' to search for the rest of the points
            fit = np.polyfit(y_list, x_list, 2)

    #Create an image to show which points were used
    image_line = np.zeros((300, 300))
    for x, y in zip(x_list, y_list):
        image_line[y,x] = 1

    #Smooth
    fit = smooth_fit( fit, old_fit)

    return fit, image_line

def smooth_fit(new_fit, old_fit, alpha = 0.8):
    beta = 1 - alpha
    fit = list([n*alpha+o*beta for n,o in zip(new_fit,old_fit)])
    return fit

#Draws a lane line on an image of specified thickness
def make_image_overlay_line( image_overlay, fit, channel = 0, thickness = 5):
    y_fit_list =  list(range(300))
    x_fit_list = [int(fit[0]*y**2 + fit[1]*y + fit[2]) for y in y_fit_list]
    for x, y in zip(x_fit_list, y_fit_list):
        if( y > 0 and y < 300 and x > 0 and x < 300):
            image_overlay[y-thickness:y+thickness,x-thickness:x+thickness,channel] = 1
    return image_overlay

#Fills the region between the image
def make_image_overlay_fill( image_overlay, l_fit, r_fit):
    y_fit_list =  list(range(300))

    l_x_fit_list = [int(calcPoint( l_fit, y )) for y in y_fit_list]
    r_x_fit_list = [int(calcPoint( r_fit, y )) for y in y_fit_list]

    #Get the list of the points for the left and right lane lines
    l_fit_points = zip(l_x_fit_list, y_fit_list)
    r_fit_points = zip(r_x_fit_list, y_fit_list)

    #Loop through and create a list of np array rects to fill
    all_rects= []
    #Don't draw until the prev_points are set
    draw = False
    for point1, point2 in zip(list(l_fit_points), list(r_fit_points)):
        if draw == True:
            #Create a rectangle 
            rect = np.array([point1, point2, point2_prev, point1_prev])
            all_rects.append(rect)
        #Current points are saved as the next iteration's previous points
        point1_prev = point1
        point2_prev = point2
        draw = True
    cv2.fillPoly(np.asarray(image_overlay), all_rects, (0,1,0))#1)
    return image_overlay

def make_image_overlay( l_fit, r_fit, thickness = 5):
    #Create a blank image
    image_overlay = np.zeros((300, 300, 3))
    #Draw the left lane line
    if( not fitIsZero( l_fit ) ):
        image_overlay = make_image_overlay_line(image_overlay, l_fit, channel = 0, thickness=thickness)
    #Draw the right lane line
    if( not fitIsZero( r_fit ) ):	
        image_overlay = make_image_overlay_line(image_overlay, r_fit, channel = 2, thickness=thickness)
    #Fill the middle
    if( not fitIsZero( l_fit ) and not fitIsZero( r_fit )):	
        image_overlay = make_image_overlay_fill(image_overlay, l_fit, r_fit)
    return image_overlay

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    #print("COMPOSITE")
    #print(img.shape)
    #print(initial_img.shape)

    img = img.astype('uint8')
    initial_img = initial_img.astype('uint8')

    return cv2.addWeighted(initial_img, α, img, β, λ)

def fitIsZero( fit, zero=0.001 ):
    #Used to determine if the poly fit is uninitialized
    return fit[2] < zero and fit[1] < zero and fit[0] < zero

def lineOutOfBounds( fit, y_top = 200, y_bottom = 300, x_left = 1, x_right = 298 ):
    #Checking if any part of the line goes out of bounds
    #By default, only checks the botom third of the image
    for y in range( y_top, y_bottom):
        test = pointOutOfBounds( fit, y, x_left, x_right) 
        if test :
            return True
    return False

def pointOutOfBounds( fit, y, x_left = 1, x_right = 298 ):
	#Checks to see if point y is out of bounds
	x = calcPoint(fit, y)
	if x < x_left or x > x_right:
		return True
	return False

def calcPoint( fit, y ):
	#Calculates x for y on a given poly fit.
	return fit[0]*y**2 + fit[1]*y + fit[2]

def calcCurve( fit, xm_per_pix = 3.7/700, ym_per_pix = 30/720 ):
    #Create y and calculate x
    y_fit_list =  list(range(300))
    x_fit_list = [int(calcPoint(fit, y)) for y in y_fit_list]

    #Convert to meter space from pixel space
    y_fit_list_m = [y*ym_per_pix for y in y_fit_list]
    x_fit_list_m = [x*xm_per_pix for x in x_fit_list]

    # Fit new polynomial to x,y in meter space
    fit_m = np.polyfit(y_fit_list_m, x_fit_list_m, 2)

    # Calculate the curvature
    y = 300 
    A = fit_m[0]
    B = fit_m[1]
    curve=(1+(2*A*y+B)**2)**(3/2)/abs(2*A)

    return curve

def calcAvgCurve( l_fit, r_fit, xm_per_pix = 3.7/700, ym_per_pix = 30/720 ):
	#Calculate the curvature for the left and right and returns the average
	l_curve = calcCurve(l_fit, xm_per_pix, ym_per_pix)
	r_curve = calcCurve(r_fit, xm_per_pix, ym_per_pix)
	return ( l_curve + r_curve ) / 2.0

def calcLeftRight( l_fit, r_fit, xm_per_pix = 3.7/700 ):
	y = 300

	#Calculate the origin of the left and right lane lines
	l_x = calcPoint(l_fit, y)
	r_x = calcPoint(r_fit, y)

	#Take the median
	x = (l_x + r_x) / 2.0

	#Subtract from 150 (300/2)
	x_offset = x - 150

	#Convert to meter space
	x_offset_m = x_offset * xm_per_pix

	return x_offset_m

def createCurveString(l_fit, r_fit):
	#Calculate and format the curvature string
	curve = calcAvgCurve( l_fit, r_fit ) 
	curveString = "Radius of Curvature = " + "{0:.1f}".format(curve) + " m"
	return curveString

def createOffsetString(l_fit, r_fit):
	#Calculate and format the offset string
	offset = calcLeftRight( l_fit, r_fit ) 
	if (offset < 0):
		offset_l_r = "right"
	else:
		offset_l_r = "left"
	offsetString = "Vehicle is " + "{0:.3f}".format(abs(offset)) + " m " + offset_l_r + " of center"
	return offsetString


def process( image ):
	global r_fit
	global l_fit
	global objpoints
	global imgpoints

	#cv2.imwrite("image.jpg", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
	
	#Correct for camera lense distortion
	image_undist = undistort( image, objpoints, imgpoints )
	#cv2.imwrite("image_undist.jpg", cv2.cvtColor(image_undist, cv2.COLOR_BGR2RGB) )
	
	#Image analysis - sum of certain channels and gradients
	image_comb = thresh_pipeline( image_undist )
	#cv2.imwrite("image_comb.jpg", image_comb*16)
	
	#Threshold the image
	image_thresh = thresh(image_comb, thresh = (6, 16))
	#cv2.imwrite("image_thresh.jpg", image_thresh*255)

	#Warp the image to the plane of the road
	M, Minv = calcwarp(image_undist, size = 300)
	image_warped = warp(image_thresh, M, (300,300))
	#cv2.imwrite("image_warped.jpg", image_warped*255)
	
	#Use the histogram method to reset if necessary
	l_hist, r_hist = histogram_points(image_warped)
	if(l_fit[0] * r_fit[0] < -1): #Curving opposite directions
		r_fit = [0,0,r_hist]	
		l_fit = [0,0,l_hist]
	else:
		#Little bit of sanity checking.. If the fits are unititialized or bad, reset
		if( lineOutOfBounds(l_fit) ): 
			l_fit = [0,0,l_hist]
		if( lineOutOfBounds(r_fit) ):
			r_fit = [0,0,r_hist]		

	#Use the previous poly fit as guides to calculate the new lane lines
	#Only caveat is - I don't do any smoothing
	l_fit, image_l_fit = run_fit(image_warped, l_fit, 100, 50, 0, 149)
	r_fit, image_r_fit = run_fit(image_warped, r_fit, 100, 50, 150, 299)
	#cv2.imwrite("image_l_fit.jpg", image_l_fit*255)
	#cv2.imwrite("image_r_fit.jpg", image_r_fit*255)

	#Draw the lane lines based on the poly fits
	image_overlay = make_image_overlay( l_fit, r_fit, thickness = 9)
	#cv2.imwrite("image_overlay.jpg", image_overlay*255)
	
	#Unwarp the lane lines back into the plane of the image
	image_overlay_unwarped = warp(image_overlay, Minv, (1280,720))
	#cv2.imwrite("image_overlay_unwarped.jpg", image_overlay_unwarped*255)
	
	#Combine the original image with the unwarped overlay
	image_composite = weighted_img(255*image_overlay_unwarped, image_undist)
	#cv2.imwrite("image_composite.jpg", cv2.cvtColor(image_composite, cv2.COLOR_BGR2RGB))
	
	#Create some text for the HUD and put it on the image
	curve_string = createCurveString(l_fit, r_fit)	
	offset_string = createOffsetString(l_fit, r_fit)
	image_text = cv2.putText(image_composite, curve_string, (10,60),
		cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), thickness = 3 )
	image_text = cv2.putText(image_composite, offset_string, (10,120),
		cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), thickness = 3 )

	#Save me an image because I like watching
	cv2.imwrite("image_text.jpg", cv2.cvtColor(image_text, cv2.COLOR_BGR2RGB))
	
	return image_text


# Make a list of calibration images and calibrate
calibration_fnames = glob.glob('camera_cal/calibration*.jpg')
objpoints, imgpoints = calibrate( calibration_fnames, nx=6, ny=9)

# Initialize fit vectors
l_fit = [0.0,0.0,0.0]
r_fit = [0.0,0.0,0.0]



# Make a video!
#inputVideoName = "project_video.mp4"
#inputClip = VideoFileClip(inputVideoName)
#outputClip = inputClip.fl_image(process)
#outputVideoName = inputVideoName.split('.')[0] + "_output.mp4"
#outputClip.write_videofile(outputVideoName, audio=False)

In [2]:
import matplotlib.image as mpimg
import numpy as np
import cv2
from skimage.feature import hog
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features 
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=False, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        #Should just replace all the below with this line...
        #single_img_features(image, color_space=color_space, spatial_size=spatial_size,
        #                hist_bins=hist_bins, orient=orient, 
        #                pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel,
        #                spatial_feat=spatial_feat, hist_feat=hist_feat, hog_feat=hog_feat)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features

# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=False, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
        
        
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=False, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        #print(type(img))
        #print(window)
        crop_img = img[window[0][1]:window[1][1], window[0][0]:window[1][0]]
        test_img = cv2.resize(crop_img, (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows




# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy


# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list


def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap


def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap


In [3]:
# Test my process on a few sample images
test_fnames = glob.glob('test_images\\test?.jpg') 
#print(test_fnames)
#for idx, fname in enumerate( test_fnames ):
fname = 'test_images\\test1.jpg'
image = mpimg.imread( fname )
image_undist = undistort( image, objpoints, imgpoints )
image_float = np.float32(image_undist)/255.


In [4]:
car = glob.glob('vehicles/vehicles/*/*.png')
notcar = glob.glob('non-vehicles/non-vehicles/*/*.png')

In [5]:
color_space = 'HLS'
spatial_size = (32,32)
hist_bins = 32
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel='ALL'  

##Reviewer Recommendations
color_space = 'YCrCb'
orient = 11
pix_per_cell = 16
cell_per_block = 2
hog_channel = "ALL"
        
car_features = extract_features(car, color_space=color_space, spatial_size=spatial_size,
                        hist_bins=hist_bins, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel)
notcar_features = extract_features(notcar, color_space=color_space, spatial_size=spatial_size,
                        hist_bins=hist_bins, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel)

In [6]:
print(np.amax(car_features))

1.0


In [7]:
from sklearn.preprocessing import StandardScaler
if len(car_features) > 0 and len(notcar_features) > 0:
    # Create an array stack of feature vectors
    #print(len(car_features))
       
    X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
    X_scaler = StandardScaler().fit(X)
    scaled_X = X_scaler.transform(X)
                           
    car_features_scaler = StandardScaler().fit(car_features)
    scaled_car_features = car_features_scaler.transform(car_features)
    
    notcar_features_scaler = StandardScaler().fit(notcar_features)
    scaled_notcar_features = notcar_features_scaler.transform(notcar_features)
    
    #X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
    #X_scaler = StandardScaler().fit(X)
    #scaled_X = X_scaler.transform(X)
    
    
    
    # Plot an example of raw and scaled features
    #fig = plt.figure(figsize=(12,4))
    #plt.subplot(131)
    #car_ind = np.random.randint(0, len(car))
    #car_image = mpimg.imread(car[car_ind])
    #cv2.imwrite("car_image.jpg", cv2.cvtColor(car_image, cv2.COLOR_RGB2BGR) )
    #cv2.imwrite("car_image.jpg", car_features )
    #notcar_ind = np.random.randint(0, len(notcar))
    #notcar_image = mpimg.imread(notcar[notcar_ind])
    #cv2.imwrite("notcar_image.jpg", cv2.cvtColor(notcar_image, cv2.COLOR_RGB2BGR) )
    #cv2.imwrite("car_image.jpg", notcar_features )
    #plt.title('Original Image')
    #plt.subplot(132)
    #plt.plot(X[car_ind])
    #plt.title('Raw Features')
    #plt.subplot(133)
    #plt.plot(scaled_X[car_ind])
    #plt.title('Normalized Features')
    #fig.tight_layout()
    #plt.show()
else: 
    print('Your function only returns empty feature vectors...')

In [8]:
print(np.amax(X))

1.0


In [9]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
# NOTE: the next import is only valid 
# for scikit-learn version <= 0.17
# if you are using scikit-learn >= 0.18 then use this:
# from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split


# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using spatial binning of:',spatial_size,
    'and', hist_bins,'histogram bins')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC(C=0.000100)
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4)) # 0.9925
# Need to also check kappa and precision and recall

# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

Using spatial binning of: (32, 32) and 32 histogram bins
Feature vector length: 4260
7.47 Seconds to train SVC...
Test Accuracy of SVC =  0.9884
My SVC predicts:  [ 0.  1.  0.  1.  1.  1.  0.  1.  0.  1.]
For these 10 labels:  [ 0.  1.  0.  1.  1.  1.  0.  1.  0.  1.]
0.035 Seconds to predict 10 labels with SVC


In [10]:
#import math
#for Cexp in range(-10,10,1):
    #print( Cexp )
#    C =  math.pow(10, Cexp)
    #print( C )
#    svc = LinearSVC(C=C)
#    t=time.time()
#    svc.fit(X_train, y_train)
#    t2 = time.time()
    #print(round(t2-t, 2), 'Seconds to train SVC...')
    # Check the score of the SVC
#    print('With C = ', C,'Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 6)) # 0.9925


In [11]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#image = mpimg.imread('bbox-example-image.jpg')



#big_windows = slide_window(image_float, x_start_stop=[None, None], y_start_stop=[336, None], 
 #                   xy_window=(128, 128), xy_overlap=(0.8, 0.8))   


#window_img = draw_boxes(image_float*255, big_windows, color=(0, 0, 255), thick=3) 
#cv2.imwrite("big_windows.jpg", cv2.cvtColor(window_img, cv2.COLOR_BGR2RGB))

#hot_windows = search_windows(image_float, big_windows, svc, X_scaler, color_space=color_space, 
#                        spatial_size=spatial_size, hist_bins=hist_bins, 
#                        orient=orient, pix_per_cell=pix_per_cell, 
#                        cell_per_block=cell_per_block, hog_channel=hog_channel)   

#window_img = draw_boxes(image_float*255, hot_windows, color=(0, 0, 255), thick=3)                    
#cv2.imwrite("big_windows_cars.jpg", cv2.cvtColor(window_img, cv2.COLOR_BGR2RGB))

#print(len(hot_windows)*1.0/len(big_windows))

#med_windows = slide_window(image_float, x_start_stop=[None, None], y_start_stop=[336, None], 
#                    xy_window=(96, 96), xy_overlap=(0.8, 0.8))   

#window_img = draw_boxes(image_float*255, med_windows, color=(0, 0, 255), thick=2)                    
#cv2.imwrite("med_windows.jpg", cv2.cvtColor(window_img, cv2.COLOR_BGR2RGB))

#hot_windows = search_windows(image_float, med_windows, svc, X_scaler, color_space=color_space, 
#                        spatial_size=spatial_size, hist_bins=hist_bins, 
#                        orient=orient, pix_per_cell=pix_per_cell, 
#                        cell_per_block=cell_per_block, hog_channel=hog_channel)   

#window_img = draw_boxes(image_float*255, hot_windows, color=(0, 0, 255), thick=2)   
#cv2.imwrite("med_windows_cars.jpg", cv2.cvtColor(window_img, cv2.COLOR_BGR2RGB))

#print(len(hot_windows)*1.0/len(med_windows))

#small_windows = slide_window(image_float, x_start_stop=[None, None], y_start_stop=[336, None], 
#                    xy_window=(64, 64), xy_overlap=(0.8, 0.8))   

#window_img = draw_boxes(image_float*255, small_windows, color=(0, 0, 255), thick=1)                    
#cv2.imwrite("small_windows.jpg", cv2.cvtColor(window_img, cv2.COLOR_BGR2RGB))

#hot_windows = search_windows(image_float, small_windows, svc, X_scaler, color_space=color_space, 
#                        spatial_size=spatial_size, hist_bins=hist_bins, 
#                        orient=orient, pix_per_cell=pix_per_cell, 
#                        cell_per_block=cell_per_block, hog_channel=hog_channel)    

#window_img = draw_boxes(image_float*255, hot_windows, color=(0, 0, 255), thick=1)                    
#print(len(hot_windows)*1.0/len(small_windows))

#cv2.imwrite("small_windows_cars.jpg", cv2.cvtColor(window_img, cv2.COLOR_BGR2RGB))



In [12]:


#res = cv2.resize(img,None,fx=2, fy=2, interpolation = cv2.INTER_CUBIC)

In [13]:
#print(type(big_windows))
#windows = []
#windows.extend(big_windows)
#windows.extend(med_windows)
#windows.extend(small_windows)
#print(windows)

In [14]:
#windows = slide_window(image, x_start_stop=[None, None], y_start_stop=y_start_stop, 
#  



In [15]:


from scipy.ndimage.measurements import label

#hot_windows = search_windows(image_float, windows, svc, X_scaler, color_space=color_space, 
#                        spatial_size=spatial_size, hist_bins=hist_bins, 
#                        orient=orient, pix_per_cell=pix_per_cell, 
#                        cell_per_block=cell_per_block, hog_channel=hog_channel)                       
#window_img = draw_boxes(image_float*255, hot_windows, color=(0, 0, 255), thick=2)                    

#cv2.imwrite("windows_cars.jpg", cv2.cvtColor(window_img, cv2.COLOR_BGR2RGB))



In [16]:
#heatmap = np.zeros_like(image[:,:,0]).astype(np.float)
#heatmap = add_heat(heatmap, hot_windows )
#heatmap_thresh = apply_threshold(heatmap, 6) #threshold 5 or less hits
##heatmap = heatmap*1.0/np.amax(heatmap)
#heatmap_thresh = apply_threshold(heatmap, 0.2) #threshold 20% of the max or less
#cv2.imwrite("heat_cars.jpg", heatmap_thresh*255)

In [17]:
#labels = label(heatmap)
#labelarray = np.asarray(labels[0])
#cv2.imwrite("label.jpg", labelarray*255.0/labels[1])


In [18]:
#labels = label(heatmap)
#bboxes = []
#print(labels[1])
#for i in range(1, labels[1]+1):
    #print(i)
    #labelarray = np.asarray(labels[0])
#    res = np.where(labelarray==i)
#    bbox = ((min(res[1]), min(res[0])), (max(res[1]),  max(res[0])))
#    bboxes.append(bbox)
    #print(bbox)
#print(bboxes)
#window_img = draw_boxes(image_float*255, bboxes, color=(0, 0, 255), thick=2)                    
#print(len(windows))

#print(len(hot_windows)*1.0/len(windows))
#cv2.imwrite("windows_final.jpg", cv2.cvtColor(window_img, cv2.COLOR_BGR2RGB))

In [19]:
import math

In [28]:
def get_windows():
    windows = []
    y_start = 360
    y_stop = 630
    big_windows = slide_window(image_float, x_start_stop=[None, None], y_start_stop=[y_start, y_stop], 
                    xy_window=(112, 112), xy_overlap=(0.8, 0.8))   
    windows.extend(big_windows)
    
    med_windows = slide_window(image_float, x_start_stop=[None, None], y_start_stop=[y_start, y_stop], 
                    xy_window=(96, 96), xy_overlap=(0.8, 0.8))   
    windows.extend(med_windows)
    
    small_windows = slide_window(image_float, x_start_stop=[None, None], y_start_stop=[y_start, y_stop], 
                    xy_window=(64, 64), xy_overlap=(0.8, 0.8))   
    windows.extend(small_windows)
    
    return windows

prev_heatmap = np.zeros_like(image[:,:,0]).astype(np.float)
alpha = 0.25

def get_heatmap( image, windows ):
    global prev_heatmap
    global alpha
    
    heatmap = np.zeros_like(image[:,:,0]).astype(np.float)
    heatmap = add_heat( heatmap, windows )
    heatmap = apply_threshold(heatmap, 3) #threshold 3 or less hits
    
    if(np.amax(prev_heatmap)<1 or np.amin(prev_heatmap)>1): 
        #      math.isnan(np.amax(prev_heatmap)) or 
        #      math.isnan(np.amin(prev_heatmap))):
        prev_heatmap = heatmap
     
    #if(np.amax(heatmap)>0):
    #    heatmap = heatmap/np.amax(heatmap)
    heatmap = heatmap*(1.0-alpha) 
    #if(np.amax(prev_heatmap)>0):
    #    prev_heatmap = prev_heatmap/np.amax(prev_heatmap)
    prev_heatmap = prev_heatmap*alpha
    
    heatmap = heatmap + prev_heatmap
    #print(heatmap)
    heatmap = apply_threshold(heatmap, 3) #threshold 3 hits or less 

    prev_heatmap = heatmap
    
    return heatmap

def filter_labels(labels, heatmap):
    
    bboxes = []
    labelarray = np.asarray(labels[0])
    for i in range(1, labels[1]+1):
        
        res = np.where(labelarray==i)
        x1 = min(res[1])
        y1 = min(res[0])
        x2 = max(res[1])
        y2 = max(res[0])
        heatdx = abs(x2-x1)
        heatdy = abs(y2-y1)
        heatarea = heatdx*heatdy
        bbox = ((x1, y1), (x2,  y2))
        if(heatdx>10 and heatdy>10):
            heatmaparea = heatmap[y1:y2, x1:x2]
            heatmax = np.amax(heatmaparea)
            heatsum = np.sum(heatmaparea)
            heatmean = np.average(heatmaparea, weights=heatmaparea )
            #print(i)
            #print(heatmax)
            #print(heatsum)
            #print(heatmean)
            #print(heatarea)
            if(heatmean>0.05 and heatmax > 0.1):
                bboxes.append(bbox)
    return bboxes
        

def test(image):
    
 
    color_space = 'HLS'
    spatial_size = (32,32)
    hist_bins = 32
     
    #Reviewer Recommendations
    color_space = 'YCrCb'
    orient = 11
    pix_per_cell = 16
    cell_per_block = 2
    hog_channel = "ALL"
    
    
    image_undist = undistort( image, objpoints, imgpoints )
    image_float = np.float32(image_undist)/255.
     
    windows = get_windows( )
    
    hot_windows = search_windows(image_float, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, hog_channel=hog_channel)  
    
    window_img_1 = draw_boxes(image_float*255, hot_windows, color=(0, 0, 255), thick=2)                    
    cv2.imwrite("final_windows.jpg", cv2.cvtColor(window_img_1, cv2.COLOR_BGR2RGB))
    
    heatmap = get_heatmap( image_float, hot_windows )
    
    heatmapimage = heatmap*255/max(np.amax(heatmap),1)
    cv2.imwrite("final_heatmap.jpg", heatmapimage)
    
    labels = label(heatmap)
    labelarray = np.asarray(labels[0])
    labelimage = labelarray*255.0/labels[1]
    cv2.imwrite("final_label.jpg", labelimage)
    
    bboxes = filter_labels(labels, heatmap )
        
    window_img_2 = draw_boxes(image_float*255, bboxes, color=(0, 0, 255), thick=2)                    
    cv2.imwrite("final_output.jpg", cv2.cvtColor(window_img_2, cv2.COLOR_BGR2RGB))
    
    #a = cv2.cvtColor(window_img_1, cv2.COLOR_BGR2RGB)
    #b = cv2.cvtColor(heatmapimage.astype('uint'),cv2.COLOR_GRAY2RGB)
    #c = cv2.cvtColor(labelimage,cv2.COLOR_GRAY2RGB)
    #d = cv2.cvtColor(window_img_2, cv2.COLOR_BGR2RGB)
    
    #debug_img = np.vstack((a,b,c,d))
    #cv2.imwrite("final_debug.jpg", debug_img)
    
    return window_img_2

In [29]:
#alpha = 0
# Test my process on a few sample images
#test_fnames = glob.glob('test_images\\test?.jpg') 
#print(test_fnames)
#for idx, fname in enumerate( test_fnames ):
fname = 'test_images\\test1.jpg'
    #print(fname)
image = mpimg.imread( fname )
    #image_output = test(image)
image_undist = undistort( image, objpoints, imgpoints )
image_float = np.float32(image_undist)/255.
     
windows = get_windows( )
    
hot_windows = search_windows(image_float, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, hog_channel=hog_channel)  
    
window_img_1 = draw_boxes(image_float*255, hot_windows, color=(0, 0, 255), thick=2)                    
cv2.imwrite("final_windows.jpg", cv2.cvtColor(window_img_1, cv2.COLOR_BGR2RGB))
    
heatmap = get_heatmap( image_float, hot_windows )
heatmapimage = heatmap*255/np.amax(heatmap)
cv2.imwrite("final_heatmap.jpg", heatmapimage)
    
labels = label(heatmap)
labelarray = np.asarray(labels[0])
labelimage = labelarray*255.0/labels[1]
cv2.imwrite("final_label.jpg", labelimage)

bboxes = filter_labels(labels, heatmap )

window_img_2 = draw_boxes(image_float*255, bboxes, color=(0, 0, 255), thick=2)                    
cv2.imwrite("final_output.jpg", cv2.cvtColor(window_img_2, cv2.COLOR_BGR2RGB))
    
    #a = cv2.cvtColor(window_img_1, cv2.COLOR_BGR2RGB)
    #b = cv2.cvtColor(heatmapimage.astype('uint'),cv2.COLOR_GRAY2RGB)
    #c = cv2.cvtColor(labelimage,cv2.COLOR_GRAY2RGB)
    #d = cv2.cvtColor(window_img_2, cv2.COLOR_BGR2RGB)
    
    #debug_img = np.vstack((a,b,c,d))
    #cv2.imwrite("final_debug.jpg", debug_img)
    
    #return window_img_2

True

In [26]:

prev_heatmap = np.zeros_like(image[:,:,0]).astype(np.float)
inputVideoName = "test_video.mp4"
inputClip = VideoFileClip(inputVideoName)
outputClip = inputClip.fl_image(test)
outputVideoName = inputVideoName.split('.')[0] + "_output_final.mp4"
outputClip.write_videofile(outputVideoName, audio=False)


[MoviePy] >>>> Building video test_video_output_new_4.mp4
[MoviePy] Writing video test_video_output_new_4.mp4



 97%|██████████████████████████████████████▉ | 38/39 [6:27:25<00:27, 27.78s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_video_output_new_4.mp4 



In [27]:

prev_heatmap = np.zeros_like(image[:,:,0]).astype(np.float)
inputVideoName = "project_video.mp4"
inputClip = VideoFileClip(inputVideoName)#.subclip(10, 11)
outputClip = inputClip.fl_image(test)
outputVideoName = inputVideoName.split('.')[0] + "_output_final.mp4"
outputClip.write_videofile(outputVideoName, audio=False)

[MoviePy] >>>> Building video project_video_output_new_4.mp4
[MoviePy] Writing video project_video_output_new_4.mp4



  0%|                                       | 2/1261 [00:56<9:34:52, 27.40s/it]

KeyboardInterrupt: 

In [ ]:
print(type(heatmap))

In [ ]:
from skimage import data, color, exposure
car_ind = np.random.randint(0, len(car))
car_image = mpimg.imread(car[car_ind])
#print(type(car_image))
#print(car_image.shape)
car_image_hls = cv2.cvtColor(car_image, cv2.COLOR_RGB2HLS)
for channel in range(car_image_hls.shape[2]):
    fd, hog_car_image = hog(car_image_hls[:,:,channel], orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=True, feature_vector=False)
    #print(hog_car_image)
    hog_car_image_rescaled = exposure.rescale_intensity(hog_car_image, in_range=(0, 255))
    #print(hog_car_image_rescaled)
    cv2.imwrite("hog_car_image_"+str(channel)+".jpg", hog_car_image*255/np.amax(hog_car_image))
    
from skimage import data, color, exposure
car_ind = np.random.randint(0, len(notcar))
car_image = mpimg.imread(notcar[car_ind])
#print(type(car_image))
#print(car_image.shape)
car_image_hls = cv2.cvtColor(car_image, cv2.COLOR_RGB2HLS)
for channel in range(car_image_hls.shape[2]):
    fd, hog_car_image = hog(car_image_hls[:,:,channel], orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=True, feature_vector=False)
    #print(hog_car_image)
    hog_car_image_rescaled = exposure.rescale_intensity(hog_car_image, in_range=(0, 255))
    #print(hog_car_image_rescaled)
    cv2.imwrite("hog_notcar_image_"+str(channel)+".jpg", hog_car_image*255/np.amax(hog_car_image))
    
    #cv2.imwrite("car_image.jpg", cv2.cvtColor(car_image, cv2.COLOR_RGB2BGR) )
    #cv2.imwrite("car_image.jpg", car_features )
    #notcar_ind = np.random.randint(0, len(notcar))
    #notcar_image = mpimg.imread(notcar[notcar_ind])
    #cv2.imwrite("notcar_image.jpg", cv2.cvtColor(notcar_image, cv2.COLOR_RGB2BGR) )
    #cv2.imwrite("car_image.jpg", notcar_features )
    #plt.title('Original Image')
    #plt.subplot(132)
    #plt.plot(X[car_ind])
    #plt.title('Raw Features')
    #plt.subplot(133)
    #plt.plot(scaled_X[car_ind])
    #plt.title('Normalized Features')
    #fig.tight_layout()
    #plt.show()